# Modelagem e Simulação 


# Quem será o campeão? 

Uma das paixões no Brasil é o futebol. A cada campeonato, diversos especialistas analisam elencos, táticas, etc., para tentar prever quem será o campeão. Neste *notebook*, iremos construir um simulador simples para tentar prever qual time vai ser o campeão. 

A simulação usa o [ranqueamento global de clubles](https://projects.fivethirtyeight.com/global-club-soccer-rankings/), mantido pelo *site* de predições [fivethirdeight](https://fivethirtyeight.com/). Esse ranqueamento é atualizado frequentemente, e pode ser baixado nesse [link](https://data.fivethirtyeight.com/#soccer-spi). Nessa simulação, usaremos uma cópia baixada no dia 09 de novembro de 2020, e que pode ser vista [nesse endereço](https://drive.google.com/file/d/1XJpC_XY5Pf-AIhBvKrCQVyuUa_357MSf/view?usp=sharing).

Vamos carregar essa base usando o pandas. Além dela, vamos carregar  outra bibliotecas que usaremos: numpy.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
clubes = pd.read_csv("https://drive.google.com/u/3/uc?id=1XJpC_XY5Pf-AIhBvKrCQVyuUa_357MSf&export=download",index_col='name')

Analisando uma amostra dos dados, como mostrado na próxima célula, podemos observar que, para cada clube, ela contém o rank atual (`rank`), o rank do ano anterior (`prev_rank`), o nome do clube (`name`), o seu potencial ofensivo (`off`) e defensivo (`def`), e o potencial de pontuação (`spi`).



In [ ]:
clubes.sample(10) # mostra 10 linhas aleatorias da tabela

,rank,prev_rank,league,off,def,spi
name,,,,,,
Stevenage,632,631,English League Two,0.20,2.08,7.69
CFR 1907 Cluj,183,179,UEFA Europa League,1.50,1.26,51.38
Espanyol,88,98,Spanish Segunda Division,1.55,0.84,62.65
Arsenal,22,19,Barclays Premier League,2.10,0.56,79.46
Molde,168,156,Norwegian Tippeligaen,1.89,1.57,52.84
Rangers,66,79,Scottish Premiership,1.83,0.91,66.65
Rodez,537,532,French Ligue 2,0.71,1.75,22.29
Crewe Alexandra,600,595,English League One,0.43,1.89,14.19
Exeter City,589,586,English League Two,0.60,2.04,15.99


## Selecionando um clube

Podemos selecionar um clube específico usando o comando `.loc`, e colocando o nome do clube entre colchetes. Por exemplo, para selecionar o Barcelona, podemos fazer:

In [ ]:
clubes.loc['Barcelona']

## Potencial ofensivo e defensivo


A nossa simulação irá se basear no **potencial ofensivo** e **defensivo** de cada clube. 

Sobre potencial ofensivo `off` temos que:
* é o número médio de gols que se espera que um clube marque enfrentado um clube com potencial defensivo igual a 1. 
* Por exemplo, o valor de off=1,5 para um "clube A" indica que se ele jogar 10 partidas contra um "clube B" com def=1, espera-se que o "clube A" marque 15 gols em um campo neutro (marcará 1,5 gol em média). 

Sobre potencial defensivo `def` temos que:
* é o número de gols que se espera que um clube sofra, enfrentando um clube com potencial ofensivo = 1.
* Por exemplo, o valor de def=1,5 de um "clube B" indica que se ele jogar 10 partidas contra um "clube A" com off=1, em um campo neutro, o "clube B" sofrerá 15 gols (sofrerá 1,5 gol em média).


A simulação do site [fivethirdeight](https://projects.fivethirtyeight.com/soccer-predictions/brasileirao/) é bem mais complexa, e contempla diversos fatores adicionais:
* se o time está jogando em seu estádio ou fora dele;
* a dificuldade do campeonato;
* a incerteza a respeito do potencial ofensivo e defensivo
* a evolução desses potenciais ao longo do tempo. 

Além disso, ela é refeita com os resultados das partidas já disputadas, simulando o restante do campeonato. Mas a nossa simulação é uma boa aproximação. 

# Campeonato Brasileiro
Vamos filtrar apenas os 20 clubes da Série A do Campeonato Brasileiro. 

In [ ]:
brasileiro = clubes.query("league == 'Brasileiro Série A'")
brasileiro

,rank,prev_rank,league,off,def,spi
name,,,,,,
Flamengo,55,50,Brasileiro Série A,2.00,0.96,68.72
Grêmio,83,91,Brasileiro Série A,1.60,0.85,63.57
São Paulo,89,86,Brasileiro Série A,1.51,0.81,62.57
Palmeiras,100,116,Brasileiro Série A,1.56,0.92,60.88
Atletico Mineiro,105,100,Brasileiro Série A,1.69,1.04,60.65
Internacional,123,113,Brasileiro Série A,1.44,0.92,58.15
Santos,154,150,Brasileiro Série A,1.38,1.02,54.27
Atlético Paranaense,178,193,Brasileiro Série A,1.21,0.95,51.83
Fluminense,182,177,Brasileiro Série A,1.27,1.02,51.46


# Simulando jogos

Para a simulação de jogos, vamos usar o **potencial ofensivo e defensivo** dos clubes envolvidos em um jogo. 

**Para simular o número de gols de cada equipe, vamos usar a distribuição de Poisson** (https://pt.wikipedia.org/wiki/Distribui%C3%A7%C3%A3o_de_Poisson). Em linhas gerais, essa distribuição modela eventos aleatórios com uma taxa $\lambda$ conhecida. 

No caso de nossa simulação, usaremos duas distribuições de Poisson, sendo que cada uma delas é usada para simular o número de gols de cada time. Por exemplo, se os times $A$ e $B$ estão jogando, a taxa do time $A$ ($\lambda_A$)  será a multiplicação entre o potencial ofensivo do time $A$ e do potencial defensivo do time $B$, como mostrado na equação:

$$ \lambda_A = \text{off}_A \times \text{def}_B .$$

Já a taxa do time $B$ ($\lambda_B$)  será a razão entre o potencial ofensivo do time $B$ e do potencial defensivo do time $A$, como mostrado na equação:

$$ \lambda_B = \text{off}_B \times  \text{def}_A .$$




Dessa maneira, se um time $A$ com um potencial ofensivo de 2 enfrentar um time B com potencial defensivo 1,5, o valor de $\lambda_A = 3$, ou seja, espera-se que o time $A$ marque 3 gols em média por jogo contra o time $B$. Já se o time $B$ tem um potencial ofensivo de 2, mas o time $A$ tem um pontencial defensivo de 0,5, o valor de $\lambda_B = 1$, ou seja, espera-se que o time $B$ marque apenas um gol em média por jogo contra o time $A$. 


**Em Python, podemos usar a função `random.poisson` da biblioteca `numpy` para simular o número de gols de cada time**. 

## Simulando uma  partida

Vamos receber dois times de nossa tabela e simular uma partida entre eles. 

Ou seja, iremos calcular o valor de $\lambda_A$ e $\lambda_B$ a partir do potencial ofensivo e defensivo de cada time, e calcular o número de gols de cada time.

Como visto anteriormente, podemos selecionar um clube da tabela usando o comando `.loc`, por exemplo:

In [ ]:
brasileiro.loc['Santos']

Execute o código abaixo com dois times de sua escolha. Observe que os valores de simulação podem variar se executarmos mais de uma vez, pois usamos a probabilidade na nossa simulação.

In [ ]:
timeA = input("Digite o nome de time A ")
timeB = input("Digite o nome de time B ")

lambdaA = brasileiro.loc[timeA,'off'] * brasileiro.loc[timeB,'def'] 
lambdaB = brasileiro.loc[timeB,'off'] * brasileiro.loc[timeA,'def'] 

golstimeA = np.random.poisson(lambdaA)
golstimeB = np.random.poisson(lambdaB)

print("Time A fez", golstimeA, "gols")
print("Time B fez", golstimeB, "gols")

### **Exercício:** 

O time que joga fora do seu estádio (o "visitante"), em geral, tem um potencial menor que o time que joga em seu estádio (o "mandante"). Considerando que o time $A$ é o mandante e o time $B$ é o visitante, crie uma versão do código que calcula o número de gols em uma partida em que o valor de $\lambda$ tem uma redução de 20% para o time visitante.



## Simulando campeonatos

**Vamos simular agora um campeonato em que todos os times jogam contra todos os outros times duas vezes (campeonato de pontos corridos).** Como você deve saber, o vencedor da partida ganha 3 pontos. Em caso de empate, cada time ganha um ponto. 

Inicialmente vamos criar uma banco de dados para armazenar a tabela do campeonato. Nessa tabela, vamos armazenar:

- total de **pontos** que o time conquistou no campeonato
- número de **vitórias** que o time obteve no campeonato
- numero de **empates** que o time obteve no campeonato
- número de **derrotas** que o time obteve no campeonato
- número de **gols marcados** por cada time no campeonato
- número de **gols sofridos** por cada time no campeonato
- diferença entre gols marcados e gols sofridos (**saldo de gols**) 

In [ ]:
# cria a tabela
tabela = pd.DataFrame(index=brasileiro.index,columns=["pontos", "vitórias", "empates", "derrotas", "gols marcados", "gols sofridos", "saldo de gols"])

# inicializa os valores com zero
tabela.fillna(0,inplace=True)

# mostra a tabela
tabela

**Como simular um campeonato em que todos os times jogam contra todos os outros times duas vezes (campeonato de pontos corridos)?**
Uma maneira de fazer com que todos os times joguem contra todos os outros times duas vezes é usar um ***laço duplo***. Nesse laço duplo temos laço exeterno e laço interno. Para cada valor fixo de laço externo é percorrido o laço interno. No código abaixo vamos apenas imprimir quem joga contra quem e na qual ordem, para entender melhor como funciona o laço duplo.

In [ ]:
contador_jogos=0
for timeA in brasileiro.index:   # primeiro laço - laco EXTERNO
  for timeB in brasileiro.index:     # segundo laço - laco INTERNO
  # laco externo: fixamos primeiro time - Flamengo
     # com primeiro time fixo, Flamengo, percorremos laco interno (todos os times)
  # laco externo: fixamos segundo time - Gremio
     # com segundo time fixo, Gremio, percorremos laco interno (todos os times)
  # etc
     if timeA != timeB:                  # se os times sao diferentes (o time não joga contra ele mesmo)
         contador_jogos = contador_jogos+1             # aumentamos contador de jogos por 1
         print(contador_jogos, timeA, "-", timeB)      # imprimimos o numero de jogo e os times que jogam

Note que de fato, cada time, joga contra todos os outros times duas vezes. 

Veja, por exemplo, Flamengo e Grêmio.

Primeira vez jogam quando Flamengo está fixo no laço externo (e é percorrido todo laço interno) - jogo 1.

Segunda vez jogam quando Grêmio está fixo no laço externo (e é percorrido todo laço interno) - jogo 20.

**Agora vamos simular os jogos. Vamos usar um laço duplo, e dentro de laço interno simulamos um jogo e atualizamos a tabela**: 

Mostraremos o resultado da simulação, ordenando pelo número de pontos, número de vitórias, saldo de gols e número de gols marcados.

Observe que a simulação pode variar se executarmos mais de uma vez, pois usamos a probabilidade na nossa simulação.

In [ ]:
# cria a tabela
tabela = pd.DataFrame(index=brasileiro.index,columns=["pontos", "vitórias", "empates", "derrotas", "gols marcados", "gols sofridos", "saldo de gols"])

# inicializa os valores da tabela com zero
tabela.fillna(0,inplace=True)

# laco duplo:
for timeA in brasileiro.index:   # primeiro laço - laco EXTERNO
  for timeB in brasileiro.index: # segundo laço - laco INTERNO

    if timeA != timeB: # o time não joga contra ele mesmo
      # simula uma partida e numero de gols nesta partida, entre os times TimeA e TimeB (código ja visto acima)
      lambdaA = brasileiro.loc[timeA,'off'] * brasileiro.loc[timeB,'def'] 
      lambdaB = brasileiro.loc[timeB,'off'] * brasileiro.loc[timeA,'def'] 

      golstimeA = np.random.poisson(lambdaA)
      golstimeB = np.random.poisson(lambdaB)

      # atualiza o número de gols marcados, sofridos e saldo
      # abaixo, utilizamos simbolo += para deixar codigo mais legivel. Por exemplo, x += y e equivalente a x = x+y
      # outro exemplo: tabela.loc[timeA,'pontos'] += 3   é equivalente a tabela.loc[timeA,'pontos'] = tabela.loc[timeA,'pontos'] + 3  
      tabela.loc[timeA,'gols marcados'] += golstimeA
      tabela.loc[timeB,'gols marcados'] += golstimeB

      tabela.loc[timeA,'gols sofridos'] += golstimeB
      tabela.loc[timeB,'gols sofridos'] += golstimeA

      tabela.loc[timeA,'saldo de gols'] += golstimeA - golstimeB
      tabela.loc[timeB,'saldo de gols'] += golstimeB - golstimeA

      # finalmente, atualizamos os pontos e número de vitórias, empates e derrotas
      if golstimeA > golstimeB:   # time A ganhou
        tabela.loc[timeA,'pontos'] += 3  
        tabela.loc[timeA,'vitórias'] += 1
        tabela.loc[timeB,'derrotas'] += 1
      elif golstimeA < golstimeB: # time B ganhou
        tabela.loc[timeB,'pontos'] += 3  
        tabela.loc[timeB,'vitórias'] += 1
        tabela.loc[timeA,'derrotas'] += 1
      else:                       # os times empataram
        tabela.loc[timeA,'pontos'] += 1
        tabela.loc[timeB,'pontos'] += 1
        tabela.loc[timeA,'empates'] += 1
        tabela.loc[timeB,'empates'] += 1

# o resultado da simulação, ordenando pelo número de pontos, número de vitórias, saldo de gols e número de gols marcados  
tabela.sort_values(['pontos','vitórias','saldo de gols','gols marcados'],ascending=False, inplace=True)
# imprimimos a tabela
tabela

### **Exercícios**:

1. Calcule a correlação entre o potencial ofensivo de cada time e o número de gols marcados em uma simulação. O que você pode dizer a respeito dessa correlação?

2. Calcule a correlação entre o potencial defensivo de cada time e o número de gols marcados em uma simulação. O que você pode dizer a respeito dessa correlação?

3. Calcule a correlação entre o número de pontos de uma simulação e o o seu `spi`. O que você pode dizer a respeito dessa correlação? 


### **Exercícios**:

1. Crie uma versão de campeonato que usa a função de simulação de partida que você criou anteriormente, e que reduz o $\lambda$ do time visitante em 20\%.

2. Faça uma simulação de campeonato para uma das ligas:
  - German Bundesliga (Campeonato Alemão)
  - Barclays Premier League (Campeonato Inglês)
  - Spanish Primera Division (Campeonato Espanhol)
  - Italy Serie A (Campeonato Italiano)

## Probabilidade de ser campeão

Agora vamos simular various campeonatos (por exemplo, 10) e a partir dessa simulação ver qual time deve ser o campeão (com a maior probabilidade), de acordo com o nosso modelo. 

In [ ]:
# cria a tabela
tabela = pd.DataFrame(index=brasileiro.index,columns=["pontos", "vitórias", "empates", "derrotas", "gols marcados", "gols sofridos", "saldo de gols"])

# inicializa os valores com zero
tabela.fillna(0,inplace=True)

for i in range(0,10): # repetimos simulacao de campeonato 10 vezes, e cada vez atualizamos a tabela:
  for timeA in brasileiro.index:   # primeiro laço
    for timeB in brasileiro.index: # segundo laço

      if timeA != timeB: # o time não joga contra ele mesmo
      
        # simula uma partida e número de gols nesta partida entre os times TimeA e TimeB (código já visto acima)
        lambdaA = brasileiro.loc[timeA,'off'] * brasileiro.loc[timeB,'def'] 
        lambdaB = brasileiro.loc[timeB,'off'] * brasileiro.loc[timeA,'def'] 

        golstimeA = np.random.poisson(lambdaA)
        golstimeB = np.random.poisson(lambdaB)

        # atualiza o número de gols marcados, sofridos e saldo
        # abaixo, utilizamos simbolo += para deixar codigo mais legivel. Por exemplo, x += y e equivalemte a x = x+y
        # outro exemplo: tabela.loc[timeA,'pontos'] += 3   é equivalente a tabela.loc[timeA,'pontos'] = tabela.loc[timeA,'pontos'] + 3  
        tabela.loc[timeA,'gols marcados'] += golstimeA
        tabela.loc[timeB,'gols marcados'] += golstimeB

        tabela.loc[timeA,'gols sofridos'] += golstimeB
        tabela.loc[timeB,'gols sofridos'] += golstimeA

        tabela.loc[timeA,'saldo de gols'] += golstimeA - golstimeB
        tabela.loc[timeB,'saldo de gols'] += golstimeB - golstimeA

        # finalmente, atualizamos os pontos e número de vitórias, empates e derrotas
        if golstimeA > golstimeB:   # time A ganhou
          tabela.loc[timeA,'pontos'] += 3  
          tabela.loc[timeA,'vitórias'] += 1
          tabela.loc[timeB,'derrotas'] += 1
        elif golstimeA < golstimeB: # time B ganhou
          tabela.loc[timeB,'pontos'] += 3  
          tabela.loc[timeB,'vitórias'] += 1
          tabela.loc[timeA,'derrotas'] += 1
        else:                       # os times empataram
          tabela.loc[timeA,'pontos'] += 1
          tabela.loc[timeB,'pontos'] += 1
          tabela.loc[timeA,'empates'] += 1
          tabela.loc[timeB,'empates'] += 1  

tabela.sort_values(['pontos','vitórias','saldo de gols','gols marcados'],ascending=False, inplace=True)
print(tabela)   # imprimimos tabela final ordenada    

                     pontos  vitórias  ...  gols sofridos  saldo de gols
name                                   ...                              
Flamengo                748       225  ...            460            384
Grêmio                  708       211  ...            418            282
Palmeiras               652       186  ...            478            211
São Paulo               649       185  ...            421            198
Atletico Mineiro        644       182  ...            483            206
Internacional           605       168  ...            498            129
Atlético Paranaense     563       153  ...            460             57
Santos                  556       156  ...            518             43
Fluminense              518       141  ...            525              0
Ceará                   511       138  ...            568            -32
Corinthians             499       138  ...            582            -51
Bahía                   496       135  ...         

Execute algumas vezes o código acima.

Qual time deve ter o maior número de pontos, de acordo com o nosso modelo?